# Load packages 

In [31]:
from neuromaps.parcellate import Parcellater
from neuromaps.datasets import fetch_annotation
import nibabel as nib
from nilearn.plotting import find_parcellation_cut_coords
from brainspace.null_models.moran import MoranRandomization
from scipy.spatial.distance import pdist, squareform
from neuromaps.stats import compare_images
import numpy as np
from scipy.stats import pearsonr
import pandas as pd 

# Define files for analysis

In [ ]:
# files: D2 raclopride map from neuromaps 
target_file = fetch_annotation(source='finnema2016') 

# group difference map from smoking study
path = "/Users/laurituominen/Documents/Research/test_neuromaps_subcortex"
group_diff_file = path + '/data/smoking_pctdiff.nii.gz'

# Make a parcellater with tian

In [33]:
# A parcellater with tian
parcellater_tian = Parcellater(path + '/atlas/Tian_Subcortex_S4_3T_1mm.nii.gz','MNI152')

# get the ROI names 
with open(path+'/atlas/Tian_Subcortex_S4_3T_label.txt', 'r') as f:
    tianlabels = [line.strip() for line in f]

# parcellate the maps
group_diff = parcellater_tian.fit_transform(group_diff_file, 'MNI152', True).squeeze()
target_map = parcellater_tian.fit_transform(target_file, 'MNI152', True).squeeze()

# create a dataframe 
df = pd.DataFrame({'Region': tianlabels, 'GroupDiff': group_diff, 'Target': target_map})
print(df)

print(pearsonr(group_diff, target_map))


            Region  GroupDiff    Target
0   HIP-head-m1-rh   0.285051  2.742955
1   HIP-head-m2-rh   2.563795  2.074309
2      THA-VAip-rh   2.713799  2.673070
3      THA-VAia-rh   2.341662  1.988697
4    HIP-head-l-rh   0.093814  3.041535
5      HIP-body-rh   3.253509  2.336727
6      HIP-tail-rh   6.098016  1.850131
7       THA-VPm-rh   4.096924  3.259965
8       THA-VPl-rh   5.057504  2.530140
9       THA-VAs-rh   6.228715  1.858935
10      THA-DAm-rh   9.196893  2.394477
11      THA-DAl-rh   9.958902  1.966195
12       PUT-VA-rh  -0.023123  5.269799
13       PUT-DA-rh   2.749125  4.692276
14       PUT-VP-rh   0.925698  3.383032
15       PUT-DP-rh   6.161201  3.704532
16       CAU-VA-rh   3.546787  3.479532
17       CAU-DA-rh   4.900630  3.585401
18     CAU-body-rh   4.724525  2.680541
19     CAU-tail-rh   5.130054  1.827581
20         lAMY-rh  -4.144912  3.508819
21         mAMY-rh  -1.777550  3.196119
22       THA-DP-rh   6.624034  2.524873
23    NAc-shell-rh   2.494896  4.804407


In [35]:

nnull = 1000
coords  = pd.read_csv(path+'/atlas/Tian_Subcortex_S4_3T_COG.txt', sep='\t', header=None).to_numpy()

eu = squareform(pdist(coords)) # calculate euclidean distance
euinv = eu.astype('float64') # inverse (smaller number if regions are further apart)
np.fill_diagonal(euinv, 1)
euinv **= -1

mrs = MoranRandomization(n_rep=nnull, joint=True, tol=1e-6, random_state=1234)
nullmaps = mrs.fit(euinv).randomize(group_diff)
nullmaps = nullmaps.T

In [36]:
corr, pval = compare_images(group_diff, target_map, nulls=nullmaps)
print("Correlation: ", corr, " P-value: ", pval)


Correlation:  -0.26942762012315874  P-value:  0.4825174825174825
